In [6]:
import requests 
import pandas as pd
import numpy as np

In [ ]:
TOKEN =  'My API Token'
HEADERS = {'Authorization' : f'token {TOKEN}'}

In [8]:
users = []
page = 1
while True:
    url = f'https://api.github.com/search/users?q=location:Chicago+followers:>100&page={page}'
    response = requests.get(url, headers=HEADERS)
    data = response.json()
    users.extend(data['items'])
    if 'next' not in response.links:
        break
    page +=1
#users

In [ ]:
first = users[0]
a = first.get('login')
a

For each user URL

In [10]:
df_user = pd.DataFrame(columns=['login','name','company','location','email','hireable','bio','public_repos','followers','following','created_at'])

In [11]:
for i in users:
    l1=[]
    a = i.get('login')
    url = f"https://api.github.com/users/{a}"
    a = requests.get(url, headers= HEADERS).json()
    l1 = [a['login'],a['name'],a['company'],a['location'],a['email'],a['hireable'],a['bio'],a['public_repos'],a['followers'],a['following'],a['created_at']]
    df_user = pd.concat([pd.DataFrame([l1], columns=df_user.columns), df_user], ignore_index=True)

In [ ]:
df_user

In [ ]:
df_user['company'] = (df_user['company'].str.strip().str.lstrip('@').str.upper())
df_user = df_user.fillna('null')
df_user

In [14]:
df_user.to_csv('users.csv', index=False)

Now for repository data 

In [15]:
df_repos = pd.DataFrame(columns=['login','full_name','created_at','stargazers_count','watchers_count','language','has_projects','has_wiki','license_name'])

In [16]:
for j in users:
    l2=[]
    user_login = j.get('login')
    url_r = f"https://api.github.com/users/{user_login}/repos"
    b = requests.get(url_r, headers= HEADERS).json()
    for k in b:
        l2 = [user_login,k['full_name'],k['created_at'],k['stargazers_count'],k['watchers_count'],k['language'],k['has_projects'],k['has_wiki']]
        if k['license'] == None:
            l2.append('null')
        else:
            l2.append(k['license']['name'])
        df_repos = pd.concat([pd.DataFrame([l2], columns=df_repos.columns), df_repos], ignore_index=True)

In [ ]:
url_r = f"https://api.github.com/users/{user_login}/repos"
b = requests.get(url_r, headers= HEADERS).json()
b[1]['license']

In [ ]:
df_repos

In [ ]:
df_repos = df_repos.fillna('null')
df_repos


In [20]:
df_repos.to_csv('repositories.csv', index=False)

Question 1: Who are the top 5 users in Chicago with the highest number of followers? List their login in order, comma-separated.

In [ ]:
# Assuming df_repos has a 'followers' column and a 'location' column.
# First, filter the DataFrame for users located in Chicago
chicago_users = df_user[df_user['location'].str.contains("Chicago", na=False)]

# Sort by the number of followers in descending order
top_chicago_users = chicago_users.sort_values(by='followers', ascending=False).head(5)

# Get the logins of the top 5 users
top_user_logins = top_chicago_users['login'].tolist()

# Print the logins as a comma-separated string
print(', '.join(top_user_logins))


In [ ]:
top_users = df_user.sort_values(by='followers', ascending=False).head(5)
top_user_logins = top_users['login'].tolist()
print(', '.join(top_user_logins))


In [ ]:
popular_logins = list(df_user.sort_values(by='followers', ascending=False).head()['login'])
for login in popular_logins:
    print(login, end=',')

Question 2:

In [ ]:
earliest_users = df_user.sort_values(by='created_at').head(5)
earliest_user_logins = earliest_users['login'].tolist()
print(', '.join(earliest_user_logins))


Question 3:

In [ ]:
# Assuming repo_df has a 'license_name' column
# Filter out rows with missing licenses
filtered_licenses = df_repos[df_repos['license_name'].notnull()]

# Count the occurrences of each license
license_counts = filtered_licenses['license_name'].value_counts()

# Get the top 3 most popular licenses
top_licenses = license_counts.head(5)

# Print the license names as a comma-separated string
print(', '.join(top_licenses.index))


Question 4

In [ ]:
# Assuming df_users has a 'company' column
# Filter out null values in the 'company' column
filtered_companies = df_user['company'].dropna()

# Count the occurrences of each company
company_counts = filtered_companies.value_counts()

# Get the top 2 companies
top_companies = company_counts.head(2)

# Print the result
print("Top 2 companies where developers work:")
for company, count in top_companies.items():
    print(f"{company}: {count}")



Question 5

In [ ]:
# Assuming df_users has a 'language' column
# Count the occurrences of each programming language, ignoring null values
language_counts = df_repos['language'].dropna().value_counts()

# Get the most popular programming language
most_popular_count = language_counts.head(2)

# Print the result
print(f"The most popular programming language is: {most_popular_language} (Count: {most_popular_count})")


Question 6

In [ ]:
# Assuming df_users has 'created_at' and 'language' columns

# Convert 'created_at' to datetime if it's not already
df_repos['created_at'] = pd.to_datetime(df_repos['created_at'], errors='coerce')

# Filter users who joined after 2020
users_after_2020 = df_repos[df_repos['created_at'] > '2020-12-31']

# Count the occurrences of each programming language, ignoring null values
language_counts = users_after_2020['language'].dropna().value_counts()

# Get the top 3 most popular programming languages
top_languages = language_counts.head(3)

# Print the results
print("Top 3 programming languages among users who joined after 2020:")
for language, count in top_languages.items():
    print(f"{language}: {count}")


Question 7.

In [ ]:
# Assuming df_repo has 'language' and 'stargazers_count' columns

# Group by 'language' and calculate the average stargazers_count
average_stars_per_language = df_repos.groupby('language')['stargazers_count'].mean()

# Get the language with the highest average
highest_average_language = average_stars_per_language.idxmax()
highest_average_count = average_stars_per_language.max()

# Print the result
print(f"The language with the highest average number of stars per repository is: {highest_average_language} (Average Stars: {highest_average_count})")


Question 8.

In [ ]:
# Assuming df_users has 'followers' and 'following' columns
# Calculate leader_strength
df_user['leader_strength'] = df_user['followers'] / (1 + df_user['following'])

# Get the top 5 users by leader_strength
top_leaders = df_user.nlargest(5, 'leader_strength')

# List their logins in order, comma-separated
top_logins = ', '.join(top_leaders['login'].tolist())

# Print the result
print(f"Top 5 users in terms of leader_strength: {top_logins}")


Question 9.

In [ ]:
# Assuming df_users has 'followers' and 'public_repos' columns

# Calculate the correlation between followers and public repositories
correlation = df_user['followers'].corr(df_user['public_repos'])

# Print the result
print(f"The correlation between the number of followers and the number of public repositories is: {correlation}")


Question 10.

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Assuming df_users has 'followers' and 'public_repos' columns
# Prepare the data
X = df_user[['public_repos']]  # Independent variable (public repositories)
y = df_user['followers']        # Dependent variable (followers)

# Remove any rows with NaN values
X = X.dropna()
y = y[X.index]  # Keep only the indices of X that are valid

# Create and fit the model
model = LinearRegression()
model.fit(X, y)

# Get the slope (coefficient)
slope = model.coef_[0]

# Print the result
print(f"The regression slope of followers on public repositories is: {slope}")


Question 11.

In [ ]:
print("NaN values in 'following':", df_user['following'].isna().sum())

# Filter users with hireable = True
hireable_users = df_user[df_user['hireable'] == True]
print("Number of hireable users:", len(hireable_users))

# Filter users with hireable = False
non_hireable_users = df_user[df_user['hireable'] == False]
print("Number of non-hireable users:", len(non_hireable_users))

# Calculate average following for hireable users, ignoring NaN
average_hireable = hireable_users['following'].mean()
print("Average following for hireable users:", average_hireable)

# Calculate average following for non-hireable users, ignoring NaN
average_non_hireable = non_hireable_users['following'].mean()
print("Average following for non-hireable users:", average_non_hireable)

# Calculate the difference, handling NaN
difference = (average_hireable if not pd.isna(average_hireable) else 0) - \
             (average_non_hireable if not pd.isna(average_non_hireable) else 0)

# Print the results
print(f"Difference in average following (hireable - non-hireable): {difference}")

Quesition 12

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Sample DataFrame (replace this with your actual df_users DataFrame)
# df_users = pd.read_csv('your_file.csv')

# Step 1: Calculate the word count of the bio
# Using fillna to avoid errors with NaN values in the bio column
df_user['bio_word_count'] = df_user['bio'].fillna('').apply(lambda x: len(x.split()))

# Step 2: Prepare the data for regression
# Ensure to drop any rows where followers or bio_word_count are NaN
df_user = df_user.dropna(subset=['followers', 'bio_word_count'])

X = df_user[['bio_word_count']]  # Independent variable (bio word count)
y = df_user['followers']          # Dependent variable (followers)

# Step 3: Create and fit the model
model = LinearRegression()
model.fit(X, y)

# Get the slope (coefficient)
slope = model.coef_[0]

# Print the result
print(f"The regression slope of followers on bio word count is: {slope}")


Question 13

In [ ]:
import pandas as pd

# Assuming repos_df is your DataFrame containing repository data
# repos_df = pd.read_csv('your_repos_file.csv')

# Step 1: Convert the created_at column to datetime
df_repos['created_at'] = pd.to_datetime(df_repos['created_at'])

# Step 2: Filter for weekend days (Saturday = 5, Sunday = 6)
df_repos['day_of_week'] = df_repos['created_at'].dt.dayofweek
weekend_repos = df_repos[df_repos['day_of_week'].isin([5, 6])]

# Step 3: Group by user login and count the number of repositories
repos_count = weekend_repos.groupby('login').size().reset_index(name='repo_count')

# Step 4: Sort by repo_count in descending order and get the top 5
top_weekend_users = repos_count.sort_values(by='repo_count', ascending=False).head(5)

# Step 5: Get the logins as a comma-separated string
top_users_logins = ', '.join(top_weekend_users['login'].tolist())

# Print the result
print(f"Top 5 users who created the most repositories on weekends: {top_users_logins}")


Question 14

In [ ]:
import pandas as pd

# Assuming df_users is your DataFrame containing user data
# df_users = pd.read_csv('your_users_file.csv')

# Step 1: Calculate the fraction of users with email when hireable=True
hireable_users = df_users[df_users['hireable'] == True]
fraction_hireable_with_email = hireable_users['email'].notna().mean()

# Step 2: Calculate the fraction of users with email for the rest (hireable=False)
non_hireable_users = df_users[df_users['hireable'] != True]
fraction_non_hireable_with_email = non_hireable_users['email'].notna().mean()

# Step 3: Calculate the difference and round to 3 decimal places
difference = round(fraction_hireable_with_email - fraction_non_hireable_with_email, 3)

# Print the result
print(f"The difference in the fraction of users with email is: {difference}")


Question 15.

In [ ]:
import pandas as pd

# Assuming df_users is your DataFrame containing user data
# df_users = pd.read_csv('your_users_file.csv')

# Step 1: Filter out null strings and empty names, then trim whitespace
df_users = df_user[(df_user['name'].notna()) & (df_user['name'] != 'null') & (df_user['name'] != '')].copy()

# Step 2: Trim whitespace
df_users['name'] = df_users['name'].str.strip()

# Step 3: Extract surnames (last word of the name)
df_users['surname'] = df_users['name'].apply(lambda x: x.split()[-1])

# Step 4: Count occurrences of each surname
surname_counts = df_users['surname'].value_counts()

# Step 5: Determine the maximum count and filter the surnames
max_count = surname_counts.max()
most_common_surnames = surname_counts[surname_counts == max_count].index.tolist()

# Step 6: Sort the surnames alphabetically and join them into a comma-separated string
most_common_surnames.sort()
result = ', '.join(most_common_surnames)

# Print the result
print(f"The most common surname(s): {result}")
